In [2]:
import os
os.environ["OMP_NUMBER_THREADS"] = "3"

import numpy as np
from openfermion.chem import MolecularData
from openfermionpyscf import run_pyscf
import mindquantum as mq
from mindquantum import Circuit, X, RX, Hamiltonian, Simulator
from mindquantum.algorithm import generate_uccsd
import mindspore as ms
import mindspore.context as context
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer

context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

In [19]:
# dist = 1.5
# geometry = [["Li", [0.0, 0.0, 0.0 * dist]],
#             ["H" , [0.0, 0.0, 1.0 * dist]]]
# # basis = "sto3g"
# # basis = "cc-pvtz"
# basis = "6-31g"
# spin = 0
# print("Geometry: ", geometry)

n_hydrogens = 6
bond_distance = 1.3
basis = "sto3g"
geometry = [('H', (0, 0, i * bond_distance)) 
    for i in range(n_hydrogens)]
print("Geometry: ", geometry)

Geometry:  [('H', (0, 0, 0.0)), ('H', (0, 0, 1.3)), ('H', (0, 0, 2.6)), ('H', (0, 0, 3.9000000000000004)), ('H', (0, 0, 5.2)), ('H', (0, 0, 6.5))]


In [20]:
# molecule_of = MolecularData(
#     geometry, 
#     basis, 
#     multiplicity=2*spin+1
# )

molecule_of = MolecularData(
    geometry=geometry,
    charge=0,
    basis=basis,
    multiplicity=1,
    description=f"linear_r-{bond_distance}"
)

molecule_of = run_pyscf(
    molecule_of, 
    run_scf=1, 
    run_ccsd=1, 
    run_fci=1
)

print("Hartree-Fock energy: %20.16f Ha" % (molecule_of.hf_energy))
print("CCSD energy: %20.16f Ha" % (molecule_of.ccsd_energy))
print("FCI  energy: %20.16f Ha" % (molecule_of.fci_energy))

Hartree-Fock energy:  -3.1025910099104959 Ha
CCSD energy:  -3.2333624471067126 Ha
FCI  energy:  -3.2345501055621719 Ha


In [21]:
molecule_of.save()
molecule_file = molecule_of.filename
print(molecule_file)

/home/zhuyk6/.pyenv/versions/3.7.5/lib/python3.7/site-packages/openfermion/testing/data/H6_6-31g_singlet_linear_r-1.3


In [22]:
hartreefock_wfn_circuit = Circuit([
    X.on(i) for i in range(molecule_of.n_electrons)
])
print(hartreefock_wfn_circuit)

q0: ──X──

q1: ──X──

q2: ──X──

q3: ──X──

q4: ──X──

q5: ──X──


In [23]:
ansatz_circuit, \
init_amplitudes, \
ansatz_parameter_names, \
hamiltonian_QubitOp, \
n_qubits, \
n_electrons = generate_uccsd(molecule_file, th=-1)

total_circuit = hartreefock_wfn_circuit + ansatz_circuit
total_circuit.summary()
print("Number of parameters: %d" % (len(ansatz_parameter_names)))

ccsd:-3.2333624471067126.
fci:-3.234550105562172.
=============================Circuit Summary=============================
|Total number of gates  : 244266.                                       |
|Parameter gates        : 7668.                                         |
|with 405 parameters are: p0, p27, p1, p28, p2, p29, p3, p30, p4, p31...|
|Number qubit of circuit: 24                                            |
Number of parameters: 405


In [17]:
sim = Simulator("projectq", total_circuit.n_qubits)
ham = Hamiltonian(hamiltonian_QubitOp)
ham.sparse(total_circuit.n_qubits)
molecule_pqc = sim.get_expectation_with_grad(
    ham,
    total_circuit
)

from mindquantum.framework import MQAnsatzOnlyLayer

molecule_pqcnet = MQAnsatzOnlyLayer(molecule_pqc, "Zeros")

initial_energy = molecule_pqcnet()
print("Initial energy: %20.16f" % (initial_energy.asnumpy()))

Initial energy:  -2.0684099197387695


In [18]:
optimizer = ms.nn.Adagrad(
    molecule_pqcnet.trainable_params(),
    learning_rate=4e-2
)
train_pqcnet = ms.nn.TrainOneStepCell(molecule_pqcnet, optimizer)

eps = 1.e-8
energy_diff = eps * 1000
energy_last = initial_energy.asnumpy() + energy_diff
iter_idx = 0

# time it
# limit iteration
while abs(energy_diff) > eps:
    energy_i = train_pqcnet().asnumpy()
    if iter_idx % 5 == 0:
        print("Step %3d energy %20.16f" % (iter_idx, float(energy_i)))
    energy_diff = energy_last - energy_i
    energy_last = energy_i
    iter_idx += 1

print("Optimization completed at step %3d" % (iter_idx - 1))
print("Optimized energy: %20.16f" % energy_i)
print("Optimized amplitudes: ", molecule_pqcnet.weight.asnumpy())

Step   0 energy  -2.0684099197387695
Step   5 energy  -2.1329057216644287
Step  10 energy  -2.1524364948272705
Step  15 energy  -2.1555552482604980
Step  20 energy  -2.1558287143707275
Step  25 energy  -2.1558537483215332
Step  30 energy  -2.1558580398559570
Optimization completed at step  34
Optimized energy:  -2.1558587551116943
Optimized amplitudes:  [-6.28285017e-03  3.89960036e-02  6.52762511e-09  1.19726472e-01
  8.91919782e-09  3.83209549e-02 -1.48043660e-02  2.72279941e-02
  1.63974036e-02  5.71735576e-03  8.02772160e-09  6.54668082e-03
  7.83180525e-08  5.03344508e-03 -1.88782793e-02  8.68890062e-03
 -5.19739930e-03  6.99206255e-03  1.98641672e-08  5.45758614e-03
 -1.23232951e-07  7.32358871e-03 -1.76936276e-02  2.97281868e-03
  3.63270170e-09  1.32459235e-08 -1.14323027e-01 -7.11524412e-02
 -2.97644013e-03  7.26065750e-08 -5.84977045e-08  6.72285907e-08
 -3.40864621e-02 -2.92904843e-02  2.39834320e-02 -8.61504219e-08
  9.46972065e-08  1.07241448e-07  2.26549488e-02  1.3412965

In [1]:
from pyqpanda import *
import numpy as np

ModuleNotFoundError: No module named 'paddle'